<a href="https://colab.research.google.com/github/JaveriaAmjad1/Langchain-/blob/main/weather_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-google-genai requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from google.colab import userdata
import os

os.environ['WEATHER_API_KEY'] = userdata.get('WEATHER_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import Tool
from langchain.agents import initialize_agent, AgentType

In [ ]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', verbose=True)

In [ ]:
api_key = os.environ.get('WEATHER_API_KEY')

In [ ]:
import requests
def get_weather(city: str) -> str:
    """Fetch weather information for a given city."""
    api_key = os.environ.get('WEATHER_API_KEY')
    if not api_key:
        return "Weather API key not found. Please set it in the environment."

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        weather = data['weather'][0]['description']
        temp = data['main']['temp']
        return f"The current weather in {city} is {weather} with a temperature of {temp}°C."
    else:
        return f"Failed to fetch weather data for {city}. Please check the city name."



In [ ]:
weather_tool = Tool(
    name="WeatherTool",
    description="Get the current weather of a city. Input should be a city name.",
    func=lambda city: get_weather(city),
)

tools = [weather_tool]

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-9-55e851b56582>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
import requests
city = input("Enter the city name: ")
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    city_name = data.get('name', 'Unknown Location')
    country = data['sys'].get('country', 'Unknown Country')
    weather_description = data['weather'][0].get('description', 'No description available')
    temperature = data['main'].get('temp', 'N/A')
    feels_like = data['main'].get('feels_like', 'N/A')
    humidity = data['main'].get('humidity', 'N/A')


    formatted_weather = (
        f"Weather Report for {city_name}, {country}:\n"
        f" - Weather: {weather_description.capitalize()}\n"
        f" - Temperature: {temperature}°C (Feels like: {feels_like}°C)\n"
        f" - Humidity: {humidity}%"
    )
    print(formatted_weather)
else:
    # Handle errors
    print(f"Failed to fetch weather data. HTTP Status Code: {response.status_code}")
    if response.status_code == 404:
        print("Error: City not found. Please check the city name.")
    elif response.status_code == 401:
        print("Error: Invalid API key. Please verify your API key.")
    else:
        print(f"Response: {response.text}")


Enter the city name: karachi
Weather Report for Karachi, PK:
 - Weather: Haze
 - Temperature: 17.9°C (Feels like: 16.76°C)
 - Humidity: 39%
